# 脚本说明
适合做初筛 (Screening)。例如，你有一百个可能的突变点，想把那些明显不稳定的（Destabilizing, ddG > 5.0）剔除掉，脚本速度快且准确度在可接受范围内。


## 注意：
- 如果你的论文核心是生物化学实验，只是用这个脚本来辅助解释实验现象（例如：“模拟显示该突变引入了严重的 steric clash”），那么这份代码完全够格。

- 如果你的论文核心是纯计算预测，那么审稿人可能会要求你增加采样次数（nstruct），或者对比 Cartesian ddG 的结果。

# 初始化与环境配置

In [1]:
import pyrosetta
from pyrosetta import *
from pyrosetta.rosetta.core.scoring import ScoreType
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import (
    IncludeCurrent, InitializeFromCommandline, RestrictToRepacking, PreventRepacking
)
from pyrosetta.rosetta.core.select.residue_selector import (
    ResidueIndexSelector, NeighborhoodResidueSelector, NotResidueSelector
)
from pyrosetta.rosetta.protocols.relax import FastRelax
import pandas as pd
import py3Dmol

# 初始化 PyRosetta
# -ex1 -ex2: 增加侧链采样精度 (关键!)
# -use_input_sc: 保留输入结构侧链作为参考
pyrosetta.init("-ex1 -ex2 -use_input_sc -nstruct 1")

print("PyRosetta Initialized Successfully.")

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.m1 2025.47+release.8bb54e8a6dc3e1c027e4f028bdace6bd4691c823 2025-11-20T15:59:05] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python310.m1 r417 2025.47+release.8bb54e8a6d 8bb54e8a6d

# 辅助工具类 (PDB编号管理)

In [2]:
def get_pose_index(pose, chain, pdb_resnum):
    """
    将 PDB 编号 (e.g., Chain A, Residue 120) 转换为 Pose 索引 (1, 2, 3...)
    """
    pdb_info = pose.pdb_info()
    pose_index = pdb_info.pdb2pose(chain, int(pdb_resnum))
    if pose_index == 0:
        raise ValueError(f"Residue {chain}:{pdb_resnum} not found in PDB.")
    return pose_index

def get_pdb_label(pose, pose_index):
    """
    将 Pose 索引反向转换为 PDB 标签，用于打印日志
    """
    pdb_info = pose.pdb_info()
    chain = pdb_info.chain(pose_index)
    resnum = pdb_info.number(pose_index)
    aa = pose.residue(pose_index).name1()
    return f"{aa}{resnum} (Chain {chain})"

# 加载与预松弛 (Pre-Relax)

In [3]:
# 1. 加载您的 PDB 文件 (请替换为您的文件名)
input_pdb = "bhpmeh_ns.pdb" 
# 如果没有文件，可以用下面这行下载一个测试用 (e.g., 1UBQ)
# pyrosetta.toolbox.rcsb.download("1UBQ")
# input_pdb = "1UBQ_clean.pdb" # 下载后通常需要clean

pose = pose_from_pdb(input_pdb)
scorefxn = create_score_function("ref2015")

print(f"Original Energy: {scorefxn(pose):.2f} REU")

# 2. 执行 FastRelax
# 这一步可能需要几分钟，取决于蛋白大小
print("Running Pre-Relax to remove crystal artifacts (Crucial for Publication)...")
relax = FastRelax()
relax.set_scorefxn(scorefxn)
relax.apply(pose)

wt_score = scorefxn(pose)
print(f"Relaxed WT Energy: {wt_score:.2f} REU")

# 保存松弛后的 WT 结构作为后续突变的基准
pose.dump_pdb("WT_relaxed.pdb")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.327291 seconds.
core.import_pose.import_pose: File 'bhpmeh_ns.pdb' automatically determined to be of type PDB from contents.
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLN:CtermProteinFull 459
core.conformation.Conformation: Found disulfide between residues 54 89
core.conformation.Conformation: Found disulfide between residues 239 250
core.conformation.Conformation: Found disulfide between residues 363 446
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database:

True

# 核心突变算法 (Mutate - Repack - Minimize)

In [49]:

from pyrosetta.rosetta.core.pack.task.operation import (
    OperateOnResidueSubset, 
    PreventRepackingRLT, 
    RestrictAbsentCanonicalAASRLT,
    RestrictToRepackingRLT  # <--- 关键修正：必须使用 RLT 版本
)
from pyrosetta.rosetta.core.chemical import DISULFIDE
from pyrosetta.rosetta.core.conformation import break_disulfide
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import IncludeCurrent, InitializeFromCommandline
from pyrosetta.rosetta.core.select.residue_selector import (
    ResidueIndexSelector, 
    NeighborhoodResidueSelector, 
    NotResidueSelector, 
    AndResidueSelector
)

# 辅助函数保持不变
def find_disulfide_partner(pose, res_index):
    res = pose.residue(res_index)
    for i in range(1, res.n_current_residue_connections() + 1):
        partner_idx = res.connected_residue_at_resconn(i)
        if partner_idx == 0 or partner_idx == res_index: continue
        if pose.residue(partner_idx).has_variant_type(DISULFIDE):
            return partner_idx
    return None

def calculate_ddg_robust(wt_pose, chain_id, pdb_resnum, target_aa_one_letter, packing_radius=8.0):
    """
    最强健版本：显式断开二硫键 + 通过 PackerTask 强制突变
    """
    # 1. 准备
    mut_pose = wt_pose.clone()
    res_index = get_pose_index(mut_pose, chain_id, pdb_resnum)
    wt_aa_name = mut_pose.residue(res_index).name1()
    
    print(f"--- Processing {wt_aa_name}{pdb_resnum} -> {target_aa_one_letter} ---")

    # 2. 处理二硫键逻辑
    is_disulfide = mut_pose.residue(res_index).has_variant_type(DISULFIDE)
    if is_disulfide:
        print("Status: Disulfide bond detected.")
        partner_index = find_disulfide_partner(mut_pose, res_index)
        if partner_index:
            print(f"Action: Breaking disulfide bond with residue {partner_index}...")
            break_disulfide(mut_pose.conformation(), res_index, partner_index)
        else:
            print("Warning: Unpaired disulfide variant found.")

    # 3. 构建 TaskFactory
    tf = TaskFactory()
    tf.push_back(InitializeFromCommandline())
    tf.push_back(IncludeCurrent())

    # A. 目标位点：强制突变 (Design)
    target_selector = ResidueIndexSelector(res_index)
    
    # RLT 初始化修正：
    rlt = RestrictAbsentCanonicalAASRLT()
    rlt.aas_to_keep(target_aa_one_letter)
    
    force_mut_op = OperateOnResidueSubset(rlt, target_selector)
    tf.push_back(force_mut_op)

    # B. 邻居位点：允许重排 (Repack only)
    nbr_selector = NeighborhoodResidueSelector(target_selector, packing_radius, True)
    nbr_only_selector = AndResidueSelector(nbr_selector, NotResidueSelector(target_selector))
    
    # --- 关键修正点 ---
    # 使用 RestrictToRepackingRLT (Residue Level Task)
    repack_op = OperateOnResidueSubset(
        RestrictToRepackingRLT(), 
        nbr_only_selector
    )
    # ----------------
    tf.push_back(repack_op)
    
    # C. 非邻居位点：完全冻结
    not_nbr_selector = NotResidueSelector(nbr_selector)
    prevent_op = OperateOnResidueSubset(PreventRepackingRLT(), not_nbr_selector)
    tf.push_back(prevent_op)

    # 4. 执行 PackRotamers
    print(f"Action: Packing and forcing mutation to {target_aa_one_letter}...")
    packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover(scorefxn)
    packer.task_factory(tf)
    packer.apply(mut_pose)

    # 5. 验证突变
    actual_new_aa = mut_pose.residue(res_index).name1()
    if actual_new_aa != target_aa_one_letter:
        print(f"CRITICAL ERROR: Mutation failed! Pose still has {actual_new_aa}")
    else:
        print(f"Verification: Mutation successful. Residue is now {actual_new_aa}")

    # 6. Minimization
    mm = MoveMap()
    mm.set_bb(False)
    mm.set_chi(False)
    
    selection = nbr_selector.apply(mut_pose)
    for i in range(1, mut_pose.total_residue() + 1):
        if selection[i]:
            mm.set_chi(i, True)

    min_mover = pyrosetta.rosetta.protocols.minimization_packing.MinMover()
    min_mover.movemap(mm)
    min_mover.score_function(scorefxn)
    min_mover.min_type("lbfgs_armijo_nonmonotone")
    min_mover.apply(mut_pose)
    
    # 7. 计算分数
    mut_score = scorefxn(mut_pose)
    ddg = mut_score - wt_score 
    
    return {
        "Mutation": f"{wt_aa_name}{pdb_resnum}{target_aa_one_letter}",
        "ddG (REU)": ddg,
        "Mutant_Pose": mut_pose
    }

# 运行计算

In [62]:
# 示例：将 A链 第 20 号残基突变为 丙氨酸 (A)
# 请修改为您需要的参数
target_chain = "A"
target_resnum = 235 # PDB编号
target_amino_acid = "Y" 

result = calculate_ddg_robust(pose, target_chain, target_resnum, target_amino_acid)

# 打印漂亮的报表
print("-" * 30)
print(f"Result for {result['Mutation']}")
print(f"ddG: {result['ddG (REU)']:.3f} REU")
print("-" * 30)

if result['ddG (REU)'] < -1.0:
    print("结论: 该突变显著增加了蛋白稳定性 (Stabilizing)")
elif result['ddG (REU)'] > 1.0:
    print("结论: 该突变破坏了蛋白稳定性 (Destabilizing)")
else:
    print("结论: 该突变对稳定性影响中性 (Neutral)")

--- Processing S235 -> Y ---
Action: Packing and forcing mutation to Y...
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 123 rotamers at 6 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
Verification: Mutation successful. Residue is now Y
------------------------------
Result for S235Y
ddG: 5.522 REU
------------------------------
结论: 该突变破坏了蛋白稳定性 (Destabilizing)


# 交互式可视化 (Publication Quality Check)

In [63]:
import py3Dmol
from pyrosetta.rosetta.std import ostringstream

def get_pdb_string(pose):
    """
    辅助函数：将 Pose 数据转换为 PDB 格式的字符串 (在内存中操作，不写文件)
    """
    oss = ostringstream()
    # 直接使用 pose 对象的成员函数，传入内存流
    pose.dump_pdb(oss) 
    return oss.str()

# 1. 初始化视图
view = py3Dmol.view(width=800, height=400)

# 2. 加载 WT (灰色)
# 使用 get_pdb_string 获取字符串内容
wt_pdb_str = get_pdb_string(pose) 
view.addModel(wt_pdb_str, "pdb")
view.setStyle({'model': 0}, {'cartoon': {'color': 'lightgray', 'opacity': 0.7}})

# 3. 加载 Mutant (蓝色)
# 注意：确保 result 变量里有 'Mutant_Pose' (来自之前的计算结果)
if 'Mutant_Pose' in result:
    mutant_pose = result['Mutant_Pose']
    mut_pdb_str = get_pdb_string(mutant_pose)
    view.addModel(mut_pdb_str, "pdb")
    view.setStyle({'model': 1}, {'cartoon': {'color': 'blue'}})

    # 4. 聚焦突变位点并设置样式
    # 找到 Pose Index
    idx = get_pose_index(pose, target_chain, target_resnum)
    pdb_res_label = str(pose.pdb_info().number(idx))

    # 显示 WT 的侧链 (红色 stick)
    view.addStyle({'model': 0, 'resi': pdb_res_label, 'chain': target_chain}, 
                  {'stick': {'colorscheme': 'redCarbon'}})

    # 显示 Mutant 的侧链 (绿色 stick)
    view.addStyle({'model': 1, 'resi': pdb_res_label, 'chain': target_chain}, 
                  {'stick': {'colorscheme': 'greenCarbon'}})

    # 调整视角到该残基
    view.zoomTo({'resi': pdb_res_label, 'chain': target_chain})

    # 显示标签
    # Note: 这里的 name1() 返回氨基酸单字母
    label_text = f"WT:{pose.residue(idx).name1()} -> Mut:{mutant_pose.residue(idx).name1()}"
    view.addLabel(label_text, 
                  {'position': {'x':0, 'y':0, 'z':0}, 'useScreen': True, 'fontColor':'black', 'backgroundColor':'white'})

else:
    print("错误：无法在 result 中找到 Mutant_Pose，请检查 calculate_ddg 是否正确运行。")

view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.